In [42]:
# Bibliotecas utilizadas no projeto

import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from ast import literal_eval

In [43]:
# Import dos dados que serão analisados

df = pd.read_excel(r"data\dados_funks.xlsx", usecols=['Nº da ficha:', 'Título da música:', 'Canal:',
       'Nº de visualizações:', 'Nº de inscritos:', 'Data de publicação', 'MC:','DJ:', 'Duração:', 
       'Estado:', 'Cidade:','Transcrição:'])

### Verificação do tipo de dado que temos

In [44]:
df.head(3)

,Nº da ficha:,Título da música:,Canal:,Nº de visualizações:,Nº de inscritos:,Data de publicação,MC:,DJ:,Duração:,Estado:,Cidade:,Transcrição:
0,1,Pela 1 vez juntos :: MC`S. Menor do Chapa e Ma...,Roda de Funk Original,240937.0,695998.0,2014-04-28 00:00:00,Menor do Chapa e Mazinho,NaN,11:01:00,Rio de Janeiro,Rio de Janeiro,1.\tMC Novinha: - Nós somos PCM porque temos d...
1,2,Mc sadrak Papo reto bonde dos 40 pedrinhas,lucas Sousa,95042.0,892.0,2014-08-01 00:00:00,Sadrak,NaN,02:50:00,Maranhão,São Luís,Se for falar que é cv no jura nao pode mais\n\...
2,3,Bonde dos 40 :::: Mc Sadrak ((Sonho de Liberda...,Conexão B.40,3938.0,3400.0,2016-05-26 00:00:00,Sadrak,não informado,03:39:00,Maranhão,São Luís,Só soldado preparado no Jacaré\n\nChamada jorn...


In [45]:
# Tamanho do nosso arquivo de dados
df.shape

(120, 12)

### 1ª Etapa - Limpeza dos dados

In [46]:
df['texto'] = df['Transcrição:']

In [47]:
# Eliminação de tabulação escape lines
df['texto'].replace(r'\r+|\n+|\t+',' ', regex=True, inplace=True)

# Conversão do texto para letras minúsculas
df['texto'] = df['texto'].str.lower()

# Eliminação de caracteres especiais
df['texto'] = df['texto'].str.replace('\W', ' ', regex=True)

# Eliminação dos números
df['texto'] = df['texto'].str.replace('\d+', '', regex=True)

# Eliminação de espaços vazios
df['texto'] = df['texto'].str.replace('  ',' ')


In [48]:
df.loc[:,['Transcrição:', 'texto']].head()

,Transcrição:,texto
0,1.\tMC Novinha: - Nós somos PCM porque temos d...,mc novinha nós somos pcm porque temos dispos...
1,Se for falar que é cv no jura nao pode mais\n\...,se for falar que é cv no jura nao pode mais v...
2,Só soldado preparado no Jacaré\n\nChamada jorn...,só soldado preparado no jacaré chamada jornalí...
3,1.\tMas sai da frente aí seus verme\n2.\tQue o...,mas sai da frente aí seus verme que o bonde ...
4,Embratel é Primeiro Comando\n1.\tO de cinza br...,embratel é primeiro comando o de cinza brotou...


### 2ª Etapa - Processamento do texto

In [49]:
# Tokenização 
regexp = RegexpTokenizer('\w+')
df['tokens']=df['texto'].apply(regexp.tokenize)

In [50]:
df['tokens'].head()

0    [mc, novinha, nós, somos, pcm, porque, temos, ...
1    [se, for, falar, que, é, cv, no, jura, nao, po...
2    [só, soldado, preparado, no, jacaré, chamada, ...
3    [mas, sai, da, frente, aí, seus, verme, que, o...
4    [embratel, é, primeiro, comando, o, de, cinza,...
Name: tokens, dtype: object

In [51]:
# Verificação das stopwords listadas para o idioma português
print(stopwords.words('portuguese'))

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [52]:
# Eliminação das stopwords
port_stopwords = stopwords.words('portuguese')
df['tokens'] = df['tokens'].apply(lambda x: [item for item in x if item not in port_stopwords])

In [53]:
#Comparação dos processamentos feitos até agora
df.loc[:,['Transcrição:', 'texto','tokens']].head()

,Transcrição:,texto,tokens
0,1.\tMC Novinha: - Nós somos PCM porque temos d...,mc novinha nós somos pcm porque temos dispos...,"[mc, novinha, pcm, porque, disposição, pcm, po..."
1,Se for falar que é cv no jura nao pode mais\n\...,se for falar que é cv no jura nao pode mais v...,"[falar, cv, jura, nao, pode, vou, dá, papo, ma..."
2,Só soldado preparado no Jacaré\n\nChamada jorn...,só soldado preparado no jacaré chamada jornalí...,"[soldado, preparado, jacaré, chamada, jornalís..."
3,1.\tMas sai da frente aí seus verme\n2.\tQue o...,mas sai da frente aí seus verme que o bonde ...,"[sai, frente, aí, verme, bonde, vai, passar, a..."
4,Embratel é Primeiro Comando\n1.\tO de cinza br...,embratel é primeiro comando o de cinza brotou...,"[embratel, primeiro, comando, cinza, brotou, l..."


In [35]:
text_freq = df['texto']
word_count = Counter(" ".join(text_freq).split()).most_common(10)
word_frequency = pd.DataFrame(word_count, columns = ['Palavra', 'Frequência'])
print(word_frequency)

  Palavra  Frequência
0       o        1712
1       é        1609
2       e        1240
3       a        1111
4     que         937
5      de         895
6      tá         828
7      do         622
8      na         604
9     nós         570


In [81]:
df['tokens'].explode().value_counts()

tá            828
pra           537
bonde         400
vai           374
aqui          300
             ... 
dada            1
veizada         1
desespero       1
pirangajem      1
olhe            1
Name: tokens, Length: 4595, dtype: int64